### Q1.Design a Sudoku puzzle where the board consists of 81 squares, some of which are initially filled with digits from 1 to 9.  

The puzzle is to fill in all the remaining squares such that no digit appears twice in any row, column, or 3 × 3 box.  
A row, column, or box is called a unit.  

1.	Represent the Sudoku problem in  CSP by identifying the variable, domain, and constraint.
2.	Implement the problem using backtracking search, what is avg. time taken by the algorithm for 10 runs.
3.	Analyse how different fault finding algorithms such as Forward Checking, Arc consistency improve the computational time of backtracking search?
4.	Analyse how different Heuristics MRV (Minimum Remaining Values), Degree heuristic, Least Constraining Value affect the  computational time of backtracking search?


In [1]:
import numpy as np
from pprint  import pprint 
import sys,warnings,copy, random,timeit
warnings.filterwarnings("ignore", category=UserWarning)  
random.seed(42)
np.random.seed(42)
from datetime import datetime
from subprocess import Popen, PIPE

In [ ]:
def timing_the_runs(stmt=""):
    # Measure the time taken for multiple runs
    execution_times = timeit.repeat(
        stmt=stmt,  # Code to time
        repeat=10,  # Number of repetitions
        number=1,   # Number of executions per repetition
        globals=globals()
    )

    # Calculate the average time
    average_time = sum(execution_times) / len(execution_times)

    print(f"Execution times: {execution_times}")
    print(f"Average time taken: {average_time:.4f} seconds")

In [3]:
def git_commit(comment=""):
    """Commit changes to the git repository."""
    time_now = datetime.now().strftime("%Y%m%d_%H%M%S")
    cmd = f'git add * ;git commit -am "{time_now} {comment}"; git push origin Sudoku;'
    # ["git", "add", "."]
    """Run a command and return its output."""
    print(cmd)
    process = Popen(cmd, stdout=PIPE, stderr=PIPE,shell=True)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        raise RuntimeError(f"Command '{' '.join(cmd)}' failed with error: {stderr.decode().strip()}")
    return stdout.decode().strip()

In [ ]:
base  = 3
side  = base*base

# pattern for a baseline valid solution
def pattern(r,c): return (base*(r%base)+r//base+c)%side

# randomize rows, columns and numbers (of valid base pattern)
from random import sample
def shuffle(s): return sample(s,len(s)) 
rBase = range(base) 
rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ] 
cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
nums  = shuffle(range(1,base*base+1))

# produce board using randomized baseline pattern
solution = [ [nums[pattern(r,c)] for c in cols] for r in rows ]
# print(solution)
board = np.array(solution)

squares = side*side
empties = squares * 3//4
for p in sample(range(squares),empties):
    board[p//side][p%side] = 0

numSize = len(str(side))


In [6]:
from IPython.display import display, HTML

def sudoku_to_html(board):
    """
    Generates an HTML table representation of a Sudoku board with thick edges for subgrids.

    Args:
        board (list of list of int): 2D list representing the Sudoku board.
                                     Empty cells should be represented by 0 or '.'.

    Returns:
        str: HTML string for the Sudoku board.
    """
    html = """
    <style>
        table { border-collapse: collapse; font-family: Arial, sans-serif; }
        td { border: 1px solid black; height: 40px; width: 40px; text-align: center; font-size: 18px; }
        .thick-border-top { border-top: 3px solid black; }
        .thick-border-left { border-left: 3px solid black; }
    </style>
    <table>
    """

    for i, row in enumerate(board):
        html += "<tr>"
        for j, cell in enumerate(row):
            # Apply thicker borders for subgrid boundaries
            classes = []
            if i % 3 == 0 and i != 0:
                classes.append("thick-border-top")
            if j % 3 == 0 and j != 0:
                classes.append("thick-border-left")
            
            class_attr = f'class="{" ".join(classes)}"' if classes else ""
            html += f"<td {class_attr}>{cell if cell != 0 else ''}</td>"
        html += "</tr>"

    html += "</table>"
    return html

In [7]:
def print_result(solved_board_dict):
    if solved_board_dict:
        solved_board = np.zeros((9, 9), dtype=int)
        for row in range(9):
            for col in range(9):
                solved_board[row][col] = solved_board_dict[(row, col)]["value"]

        # Display the solved board
        html_output = sudoku_to_html(solved_board)
        display(HTML(html_output))
    else:
        print("No solution exists.")

In [8]:
# Generate and display the Sudoku board as HTML

display(HTML(sudoku_to_html(board)))

display(HTML(sudoku_to_html(solution)))


2,,,7,,,,,
5,,,,,,,1,
,8,,3,,,,9,
,6,,2,9,7,,,
,,,,,,,,
,,8,,,,,,
,,,9,,5,,,
,,,1,7,,,,
8,,5,4,3,,,2,


2,3,1,7,5,9,6,4,8
5,7,9,8,6,4,2,1,3
6,8,4,3,2,1,5,9,7
1,6,3,2,9,7,4,8,5
9,2,7,5,4,8,1,3,6
4,5,8,6,1,3,9,7,2
7,1,2,9,8,5,3,6,4
3,4,6,1,7,2,8,5,9
8,9,5,4,3,6,7,2,1


In [9]:
board

array([[2, 0, 0, 7, 0, 0, 0, 0, 0],
       [5, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 8, 0, 3, 0, 0, 0, 9, 0],
       [0, 6, 0, 2, 9, 7, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 8, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 9, 0, 5, 0, 0, 0],
       [0, 0, 0, 1, 7, 0, 0, 0, 0],
       [8, 0, 5, 4, 3, 0, 0, 2, 0]])

1. Represent the Sudoku problem in CSP by identifying the variable, domain, and constraint.

In [10]:
def domain_cleaner(variables_dict, variable):
    """
    Cleans the domain of the variable by removing values that are already assigned to other variables.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.
        variable (tuple): The variable (row, col) whose domain needs to be cleaned.

    Returns:
        list: The cleaned domain for the variable.
    """

    neighbour_values = [variables_dict[neighbour]["value"] for neighbour in variables_dict[variable]["Neighbours"] if variables_dict[neighbour]["value"] > 0]

    cleaned_domain = [val for val in variables_dict[variable]["domain"] if val not in neighbour_values]  
    return cleaned_domain

In [11]:
def constraints_check(variable,variables_dict, test_value):
    """
    Checks if the test value satisfies all constraints.

    Args:
        assignment (dict): A dictionary mapping variables (row, col) to their assigned values.
        variables_dict (dict): The CSP variables with their domains.
        valriable (tuple): position of the cell on the board.

    Returns:
        bool: True if all constraints are satisfied, False otherwise.
    """
    # check constraints
    neighbour_values = [variables_dict[neighbour]["value"] for neighbour in variables_dict[variable]["Neighbours"] if variables_dict[neighbour]["value"] > 0]
    if test_value in neighbour_values:
        return False
    else:
        # If all constraints are satisfied, return True
        return True

In [12]:
def represent_sudoku_csp(board):
    """
    Represents a Sudoku board as a Constraint Satisfaction Problem (CSP) with detailed variable and constraint structures.

    Args:
        board (list of list of int): A 9x9 2D list representing the Sudoku board.
                                     Empty cells are represented by 0.

    Returns:
        dict: A dictionary where each key is a tuple (row, col) representing a cell, and the value is another dictionary containing:
            - 'row' (int): The row index of the cell.
            - 'col' (int): The column index of the cell.
            - 'value' (int or None): The value assigned to the cell, or None if the cell is empty.
            - 'domain' (list of int): The possible values that can be assigned to the cell.
                                      For pre-filled cells, this is a single-value list.
            - 'constraints' (dict): A dictionary of constraint functions for the cell, including:
                - 'row': A function to enforce the row constraint.
                - 'col': A function to enforce the column constraint.
                - 'box': A function to enforce the 3x3 subgrid constraint.

    Notes:
        - This function provides a more detailed representation of the CSP, where each variable (cell) is associated with its constraints and domain.
        - The constraints ensure that no number is repeated in the same row, column, or 3x3 subgrid.
        - This representation can be used with CSP solvers to solve the Sudoku puzzle.
    """
    side = len(board)
    variables = {}

    for row in range(side):
        for col in range(side):
            
            variable = {
                # 'row': row,
                # 'col': col,
                'value': board[row][col], # initial value, zero represents not filled yet
                'domain': [board[row][col]] if board[row][col]>0 else list(range(1, 10)), # initial domains

                "Neighbours": [ (r,c) for r in range(side) for c in range(side) if row == r or col == c or (row // 3 == r // 3 and col // 3 == c // 3) ], # list of neighbours
                # "Filled_Neighbours": [(r,c) for r in range(side) for c in range(side) if (row == r or col == c or (row // 3 == r // 3 and col // 3 == c // 3)) and board[r][c]>0], # list of neighbours that are already filled
            }
            variables[(row, col)] = variable
    return variables

In [13]:
variables_dict = represent_sudoku_csp(board)
for variable in variables_dict:
    # Clean the domain of the variable
    variables_dict[variable]["domain"] = domain_cleaner(variables_dict, variable)
key = list(variables_dict.keys())[1]
print(key)
pprint(variables_dict[key])
# CSP representation of the first cell of the Sudoku board

(0, 1)
{'Neighbours': [(0, 0),
                (0, 1),
                (0, 2),
                (0, 3),
                (0, 4),
                (0, 5),
                (0, 6),
                (0, 7),
                (0, 8),
                (1, 0),
                (1, 1),
                (1, 2),
                (2, 0),
                (2, 1),
                (2, 2),
                (3, 1),
                (4, 1),
                (5, 1),
                (6, 1),
                (7, 1),
                (8, 1)],
 'domain': [1, 3, 4, 9],
 'value': 0}


2. Implement the problem using backtracking search, what is avg. time taken by the algorithm for 10 runs.

In [14]:
def new_backtracking_search(variables_dict):

    Unfilled = [ v for v in variables_dict if variables_dict[v]["value"] ==0 ]
    
    # If all variables are assigned, return the assignment
    if len(Unfilled) == 0:
        return variables_dict
    
    variable = Unfilled[0]

    # Try each value in the domain of the variable
    for test_domain_value in variables_dict[variable]['domain']:

        # Check if a test value for the domain is valid
        constraints_check_flag = constraints_check(
            variable = variable,
            variables_dict = variables_dict,
            test_value = test_domain_value)
        
        if constraints_check_flag:
            
            # Create a new variables dictionary with a test value for the  domain 
            copy_variables_dict = copy.deepcopy(variables_dict)
            
            # Recursively solve the CSP with the new assignment
            copy_variables_dict[variable]["value"] = test_domain_value
            # copy_variables_dict[variable]["Filled"] = True

            result = new_backtracking_search(variables_dict = copy_variables_dict)
            if result is not None:
                return result
    # If no value leads to a solution, return None
    return None


In [15]:
timing_the_runs(stmt="new_backtracking_search(variables_dict)")

Execution times: [4.649926492000304, 4.394447108999884, 4.638912645999881, 4.208205880000605, 4.268010464999861, 4.284024705000775, 4.136527992999618, 4.401979788999597, 4.678384071000437, 4.159955711999828]
Average time taken: 4.3820 seconds


In [16]:
%%time
print_result(new_backtracking_search(variables_dict))
# new_backtracking_search(variables_dict)

2,1,3,7,4,9,5,6,8
5,4,9,6,2,8,3,1,7
6,8,7,3,5,1,2,9,4
1,6,4,2,9,7,8,5,3
3,5,2,8,1,4,6,7,9
7,9,8,5,6,3,1,4,2
4,2,1,9,8,5,7,3,6
9,3,6,1,7,2,4,8,5
8,7,5,4,3,6,9,2,1


CPU times: user 4.46 s, sys: 0 ns, total: 4.46 s
Wall time: 4.46 s


The solved board may be different from the original solution board,  
as the input board can have multiple solutions depending on the numbers removed

In [17]:
# display(HTML(sudoku_to_html(solved_board)))
# display(HTML(sudoku_to_html(solution)))

3. Analyse how different fault finding algorithms such as Forward Checking, Arc consistency improve the computational time of backtracking search?

In [18]:
def forward_checking_arc_consistency(variables_dict, variable):
    """
    Performs forward checking by pruning the domains of unassigned variables.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.
        variable (tuple): The variable (row, col) that was just assigned a value.
        test_domain_value (int): The value assigned to the variable.

    Returns:
        bool: True if no domain becomes empty after pruning, False otherwise.
    """
    copy_variables_dict_forward_checked = copy.deepcopy(variables_dict)
    test_domain_value = copy_variables_dict_forward_checked[variable]["value"]
    for neighbour in copy_variables_dict_forward_checked[variable]["Neighbours"]:
        # Prune domains of variables in the neighbourhood that are not filled
        if copy_variables_dict_forward_checked[neighbour]["value"] == 0:
            if test_domain_value in copy_variables_dict_forward_checked[neighbour]["domain"]:
                copy_variables_dict_forward_checked[neighbour]["domain"].remove(test_domain_value)
                # If domain becomes empty, return False
                if len(copy_variables_dict_forward_checked[neighbour]["domain"]) == 0:
                    return False

    return copy_variables_dict_forward_checked

In [19]:
%%time

def forward_checking_backtracking_search(variables_dict):
    """
    Solves the CSP using backtracking search with Forward Checking.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        dict or None: A complete assignment if a solution is found, or None if no solution exists.
    """
    # Find unfilled variables
    unfilled = [v for v in variables_dict if variables_dict[v]["value"]==0]

    # If all variables are assigned, return the assignment
    if not unfilled:
        return variables_dict

    # Select the first unfilled variable
    variable = unfilled[0]

    # Try each value in the domain of the variable
    for test_domain_value in variables_dict[variable]["domain"]:

        # Create a new variables dictionary with a test value for the  domain 
        
        # Constraints check will not be required as:
        # 1. the domains are cleaned
        # 2. forward checking prempts conflicts
        
        copy_variables_dict_test_domain = copy.deepcopy(variables_dict)
        
        #set the test value to the variable
        copy_variables_dict_test_domain[variable]["value"] = test_domain_value

        # Use Forward Checking to prune the domains of unassigned variables
        # and check if any domain becomes empty 
        copy_variables_dict_forward_checked = forward_checking_arc_consistency(copy_variables_dict_test_domain, variable)
        if copy_variables_dict_forward_checked:
            # Recursively solve the CSP with the new assignment
            result = forward_checking_backtracking_search(copy_variables_dict_forward_checked)
            if result is not None:
                return result
                
    # If no value leads to a solution, return None
    return None

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 5.48 μs


In [20]:
timing_the_runs(stmt="forward_checking_backtracking_search(variables_dict)")

Execution times: [4.127870453000469, 3.852800326999386, 4.200466463999874, 4.203936342999441, 3.979076194999834, 3.6514200419996996, 3.8147202469999684, 3.830972306000149, 3.965847681000014, 3.9489771730004577]
Average time taken: 3.9576 seconds


In [21]:
%%time

print_result(forward_checking_backtracking_search(variables_dict))

2,1,3,7,4,9,5,6,8
5,4,9,6,2,8,3,1,7
6,8,7,3,5,1,2,9,4
1,6,4,2,9,7,8,5,3
3,5,2,8,1,4,6,7,9
7,9,8,5,6,3,1,4,2
4,2,1,9,8,5,7,3,6
9,3,6,1,7,2,4,8,5
8,7,5,4,3,6,9,2,1


CPU times: user 4.49 s, sys: 0 ns, total: 4.49 s
Wall time: 4.49 s


d.	Analyse how different Heuristics MRV (Minimum Remaining Values), Degree heuristic, Least Constraining Value affect the  computational time of backtracking search?

In [22]:
def select_variable_with_heuristics(variables_dict,unfilled):
    """
    Selects the next variable to assign using MRV and Degree Heuristic.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        tuple: The selected variable (row, col).
    """
    # Apply MRV (Minimum Remaining Values)
    mrv = min(unfilled, key=lambda variable: len(variables_dict[variable]["domain"]))

    # If there are ties, apply Degree Heuristic
    tied_variables_degree = {variable: 0 for variable in unfilled if len(variables_dict[variable]["domain"]) == len(variables_dict[mrv]["domain"])}
    if len(tied_variables_degree) > 1:
        # Degree Heuristic: Select the variable involved in the most constraints
        for variable in tied_variables_degree:
            for neighbour in variables_dict[variable]["Neighbours"]:
                if variables_dict[neighbour]["value"] > 0:
                    tied_variables_degree[variable] += 1
        mrv = max(tied_variables_degree, key=tied_variables_degree.get)

    return mrv



In [23]:
def order_domain_values(variable, variables_dict):
    """
    Orders the domain values of a variable using the Least Constraining Value (LCV) heuristic.

    Args:
        variable (tuple): The variable (row, col) to order domain values for.
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        list: The ordered list of domain values.
    """
    sorted_domain = variables_dict[variable]["domain"] 
    count_conflicts = { }
    for value in variables_dict[variable]["domain"]:
        # Count how many variables would be affected by assigning this value
        count_conflicts[value] = len([ neighbour for neighbour in variables_dict[variable]["Neighbours"] if variables_dict[neighbour]["value"] == 0 and value in variables_dict[neighbour]["domain"] ])

    # Sort domain values by the number of conflicts (ascending)

    sorted_list = sorted(sorted_domain, key=lambda value: count_conflicts[value])
    return sorted_list

In [24]:
def backtracking_search_with_heuristics(variables_dict):
    """
    Solves the CSP using backtracking search with MRV, Degree Heuristic, and LCV.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        dict or None: A complete assignment if a solution is found, or None if no solution exists.
    """
    # Check if all variables are assigned
    unfilled = [variable for variable in variables_dict if variables_dict[variable]["value"] ==0]
    if not unfilled:
        return variables_dict

    # Select the next variable to assign using MRV and Degree Heuristic
    variable = select_variable_with_heuristics(variables_dict,unfilled)

    # Order domain values using LCV
    for test_domain_value in order_domain_values(variable, variables_dict):
        # Check if the value satisfies all constraints
        if constraints_check(variable, variables_dict, test_domain_value):
            # Assign the value to the variable
            copy_variables_dict = copy.deepcopy(variables_dict)
            copy_variables_dict[variable]["value"] = test_domain_value

            # Recursively solve the CSP
            result = backtracking_search_with_heuristics(copy_variables_dict)
            if result is not None:
                return result

    # If no value leads to a solution, return None
    return None

In [ ]:
timing_the_runs(stmt="backtracking_search_with_heuristics(variables_dict)")

In [ ]:
%%time
print_result(backtracking_search_with_heuristics(variables_dict))

In [ ]:
git_commit(comment= "Sudoku Done within ten run timings")

In [ ]:
sys.exit()